In [ ]:
%matplotlib notebook

"""
Hydro-Nbody coupling example in AMUSE
small gaseous clump disrupted by stellar binary system

to actually model the hydro, change 
>>> gas_code.particles.add_particles(gas)
to
>>> gas_code.gas_particles.add_particles(gas)

by Alessandro Alberto Trani 14/06/2018
"""

# comment this line if you're using python3
from __future__ import (absolute_import, division, print_function)

from amuse.lab import *
from amuse.couple import bridge
from amuse.datamodel.particles import Channels
from amuse.ext.molecular_cloud import molecular_cloud
import matplotlib.pyplot as plt

conv = nbody_system.nbody_to_si(0.5 | units.MSun, 1 | units.RSun)

In [ ]:
#generate initial conditions
star=Particles()
s1 = Particle(
    mass=1 | units.MSun, 
    position=(0.5,0,0) | units.AU, 
    velocity=(0, 21, 0) | units.km/units.s)
s2 = Particle(mass=1 | units.MSun, 
              position=(-0.5, 0, 0) | units.AU, 
              velocity=(0, -21, 0) | units.km/units.s)
star.add_particle(s1)
star.add_particle(s2)

gas=molecular_cloud(
    nf=32, 
    power=4, 
    targetN=1000, 
    ethep_ratio=0.001, 
    ekep_ratio=1, 
    convert_nbody=conv).result
gas.position += (-1, 1, 0) | units.AU
gas.velocity += (10, -7, 0) | units.km/units.s

In [ ]:
#call codes and assign particles
star_code = ph4(conv)
star_code.parameters.epsilon_squared = (1e-2 | units.RSun)**2
star_code.particles.add_particles(star)

gas_code = Fi(conv) 
gas_code.parameters.epsilon_squared = (1e-2 | units.RSun)**2
gas_code.particles.add_particles(gas) # note: gas is added in particles, not in gas_particles! 
                                      #       we are not actually using hydro in order to speed up things
gas_code.parameters.integrate_entropy_flag = False
gas_code.parameters.n_smooth = 32
gas_code.parameters.periodic_box_size = 1 | units.parsec
print(gas_code.parameters)

In [ ]:
#adding channels to copy particle sets to our framework
channel = Channels()
channel.add_channel(star_code.particles.new_channel_to(star))
channel.add_channel(gas_code.particles.new_channel_to(gas))

#define an interaction timestep      
bridge_system = bridge.Bridge(timestep = 0.001 | units.yr)

#make the gas be kicked by the particles of the star code
bridge_system.add_system(gas_code, (star_code,))

#make the stars be kicked by the particles of the gas code
bridge_system.add_system(star_code, (gas_code,))

In [ ]:
#plotting
fig = plt.figure(figsize=(9, 5))
ax = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

plt.ion()
fig.show()
fig.canvas.draw()

#evolve the system
tend = 1 | units.yr
dtime = 0.001 | units.yr
time = 0 | units.yr
while time < tend:
    time += dtime
    bridge_system.evolve_model(time)
    #print("evolved to time {:s}".format(time))
    channel.copy()
    
    ax.clear()
    ax2.clear()
    ax.set_ylim(-1, 1)
    ax.set_xlim(-1, 1)
    ax.set_xlabel('x [au]')
    ax.set_ylabel('y [au]')
    ax.set_aspect('equal')
    ax2.set_ylim(-1, 1)
    ax2.set_xlim(-1, 1)
    ax2.set_xlabel('x [au]')
    ax2.set_ylabel('z [au]')
    ax2.set_aspect('equal')
    ax.scatter(star.x.value_in(units.AU), star.y.value_in(units.AU))      
    ax.scatter(gas.x.value_in(units.AU), gas.y.value_in(units.AU), c = 'red', s=0.5)     
    ax.text(0.6, 0.8, "{:s}".format(time.as_quantity_in(units.yr)))
    ax2.scatter(star.x.value_in(units.AU), star.z.value_in(units.AU))      
    ax2.scatter(gas.x.value_in(units.AU), gas.z.value_in(units.AU), c = 'red', s=0.5)     
    ax2.text(0.6, 0.8, "{:s}".format(time.as_quantity_in(units.yr)))
    fig.canvas.draw()    
bridge_system.stop()